
# Ejercicio agrupaciones y apply

Agrupar los datos por equipo y posición
Calcular la eficiencia de los jugadores ( suma de puntos (PTS), asistencias (AST) y rebotes por juego(TRB) )
Identificar al jugador más eficiente por equipo

Datasets: https://www.kaggle.com/datasets/nathanlauga/nba-games/data

In [5]:
import pandas as pd
import numpy as np
import time

path = '../data/kaggle/nba_data_processed.csv'
df = pd.read_csv(filepath_or_buffer=path, sep= ',', header=0)
df.dropna(how='all',inplace=True)
df.fillna(0,inplace=True)
print(df.shape)
qsna=df.shape[0]-df.isnull().sum(axis=0)
qna=df.isnull().sum(axis=0)
ppna=round(100*(df.isnull().sum(axis=0)/df.shape[0]),2)
aux= {'datos sin NAs en q': qsna, 'Na en q': qna ,'Na en %': ppna}
na=pd.DataFrame(data=aux)
na.sort_values(by='Na en %',ascending=False)

df.head()

(679, 29)


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Precious Achiuwa,C,23.0,TOR,55.0,12.0,20.7,3.6,7.3,0.485,...,0.702,1.8,4.1,6.0,0.9,0.6,0.5,1.1,1.9,9.2
1,Steven Adams,C,29.0,MEM,42.0,42.0,27.0,3.7,6.3,0.597,...,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
2,Bam Adebayo,C,25.0,MIA,75.0,75.0,34.6,8.0,14.9,0.540,...,0.806,2.5,6.7,9.2,3.2,1.2,0.8,2.5,2.8,20.4
3,Ochai Agbaji,SG,22.0,UTA,59.0,22.0,20.5,2.8,6.5,0.427,...,0.812,0.7,1.3,2.1,1.1,0.3,0.3,0.7,1.7,7.9
4,Santi Aldama,PF,22.0,MEM,77.0,20.0,21.8,3.2,6.8,0.470,...,0.750,1.1,3.7,4.8,1.3,0.6,0.6,0.8,1.9,9.0


In [11]:
# Agrupar los datos por equipos
grouped = df.groupby(['Tm', 'Pos'])


# Paso 1: Agrupar los datos por equipo y posición
agrupado_por_equipo_posicion = df.groupby(['Tm', 'Pos']).agg({
    'PTS': 'mean',  # Promedio de puntos
    'AST': 'mean',  # Promedio de asistencias
    'TRB': 'mean'   # Promedio de rebotes
}).reset_index()

print(agrupado_por_equipo_posicion)


      Tm Pos        PTS       AST       TRB
0    ATL   C   7.000000  0.700000  5.375000
1    ATL  PF  13.100000  1.200000  6.500000
2    ATL  PG  10.433333  4.066667  1.700000
3    ATL  SF   8.400000  1.083333  3.283333
4    ATL  SG   7.483333  1.833333  2.166667
..   ...  ..        ...       ...       ...
156  WAS   C   8.680000  1.240000  3.980000
157  WAS  PF   8.560000  1.360000  3.800000
158  WAS  PG   5.166667  2.483333  2.316667
159  WAS  SF   7.266667  1.500000  3.066667
160  WAS  SG  11.050000  2.650000  2.675000

[161 rows x 5 columns]


In [12]:
# Paso 2: Calcular la eficiencia de los jugadores
# La eficiencia se define como la suma de puntos, asistencias y rebotes por juego.
df['eficiencia'] = df['PTS'] + df['AST'] + df['TRB']
# Agrupar por equipo y posición, y calcular la eficiencia promedio por posición en cada equipo
eficiencia_por_equipo_posicion = df.groupby(['Tm', 'Pos']).apply(
    lambda x: pd.Series({
        'Eficiencia Promedio': x['eficiencia'].mean()
    })
).reset_index()
print(eficiencia_por_equipo_posicion)

      Tm Pos  Eficiencia Promedio
0    ATL   C            13.075000
1    ATL  PF            20.800000
2    ATL  PG            16.200000
3    ATL  SF            12.766667
4    ATL  SG            11.483333
..   ...  ..                  ...
156  WAS   C            13.900000
157  WAS  PF            13.720000
158  WAS  PG             9.966667
159  WAS  SF            11.833333
160  WAS  SG            16.375000

[161 rows x 3 columns]


C:\Users\asosa\AppData\Local\Temp\ipykernel_10684\1470240936.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  eficiencia_por_equipo_posicion = df.groupby(['Tm', 'Pos']).apply(


In [13]:
# Paso 3: Identificar al jugador más eficiente por equipo y posición
jugador_mas_eficiente = df.groupby(['Tm', 'Pos']).apply(
    lambda x: x.loc[x['eficiencia'].idxmax(), ['Player', 'eficiencia']]
).reset_index()

print(jugador_mas_eficiente)

      Tm Pos              Player  eficiencia
0    ATL   C        Clint Capela        23.9
1    ATL  PF        John Collins        20.8
2    ATL  PG          Trae Young        39.4
3    ATL  SF     De'Andre Hunter        21.0
4    ATL  SG     Dejounte Murray        31.9
..   ...  ..                 ...         ...
156  WAS   C  Kristaps Porziņģis        34.3
157  WAS  PF          Kyle Kuzma        32.1
158  WAS  PG        Monte Morris        19.0
159  WAS  SF         Deni Avdija        18.4
160  WAS  SG        Bradley Beal        32.5

[161 rows x 4 columns]


C:\Users\asosa\AppData\Local\Temp\ipykernel_10684\3724766706.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  jugador_mas_eficiente = df.groupby(['Tm', 'Pos']).apply(
